# SageMaker Serverless Inference


## Jakeshape Performance Predictor Setup


## Table of Contents
- Setup
- Model Training
- Deployment
    - Model Creation
    - Endpoint Configuration (Adjust for Serverless)
    - Serverless Endpoint Creation
    - Endpoint Invocation
- Cleanup

## Setup

For testing you need to properly configure your Notebook Role to have <b>SageMaker Full Access</b>.

Let's start by upgrading the Python SDK, `boto3` and AWS `CLI` (Command Line Interface) packages.

In [45]:
! pip install sagemaker botocore boto3 awscli --upgrade

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com


In [16]:
# Setup clients
import boto3

client = boto3.client(service_name="sagemaker")
runtime = boto3.client(service_name="sagemaker-runtime")

### SageMaker Setup
To begin, we import the AWS SDK for Python (Boto3) and set up our environment, including an IAM role and an S3 bucket to store our data.

In [17]:
import boto3
import sagemaker
from sagemaker.estimator import Estimator

boto_session = boto3.session.Session()
region = boto_session.region_name
print(region)

sagemaker_session = sagemaker.Session()
s3_prefix = "jakeshape-performance-predictor-model"
role = sagemaker.get_execution_role()
print(role)

training_instance_type = "ml.m5.xlarge"

us-east-2
arn:aws:iam::544485497213:role/service-role/AmazonSageMaker-ExecutionRole-20230805T194168


## Model Training

Now, we train an ML model using the XGBoost Algorithm. In this example, we use a SageMaker-provided [XGBoost](https://docs.aws.amazon.com/sagemaker/latest/dg/xgboost.html) container image and configure an estimator to train our model.

In [18]:
from sagemaker.inputs import TrainingInput
squat_path = 'data/squat_prediction'
bench_path = 'data/bench_prediction'
deadlift_path = 'data/deadlift_prediction'
bodyweight_path = 'data/bodyweight_prediction'

squat_train_data = TrainingInput(f's3://{s3_prefix}/{squat_path}/squat_train_data.csv', content_type="text/csv")
squat_test_data = TrainingInput(f's3://{s3_prefix}/{squat_path}/squat_validation_data.csv', content_type="text/csv")

bench_train_data = TrainingInput(f's3://{s3_prefix}/{bench_path}/bench_train_data.csv', content_type="text/csv") 
bench_test_data = TrainingInput(f's3://{s3_prefix}/{bench_path}/bench_validation_data.csv', content_type="text/csv")

deadlift_train_data = TrainingInput(f's3://{s3_prefix}/{deadlift_path}/deadlift_train_data.csv', content_type="text/csv")
deadlift_test_data = TrainingInput(f's3://{s3_prefix}/{deadlift_path}/deadlift_validation_data.csv', content_type="text/csv")

bodyweight_train_data = TrainingInput(f's3://{s3_prefix}/{bodyweight_path}/bodyweight_train_data.csv', content_type="text/csv")
bodyweight_test_data = TrainingInput(f's3://{s3_prefix}/{bodyweight_path}/bodyweight_validation_data.csv', content_type="text/csv")

In [19]:
model_path_squat = f"s3://{s3_prefix}/{squat_path}/model"
model_path_bench = f"s3://{s3_prefix}/{bench_path}/model"
model_path_deadlift = f"s3://{s3_prefix}/{deadlift_path}/model"
model_path_bodyweight = f"s3://{s3_prefix}/{bodyweight_path}/model"

# retrieve xgboost image
image_uri = sagemaker.image_uris.retrieve(
    framework="xgboost",
    region=region,
    version="1.0-1",
    py_version="py3",
    instance_type=training_instance_type,
)

# Configure Training Estimator
xgb_train_squat = Estimator(
    image_uri=image_uri,
    instance_type=training_instance_type,
    instance_count=1,
    output_path=model_path_squat,
    sagemaker_session=sagemaker_session,
    role=role,
)
xgb_train_bench = Estimator(
    image_uri=image_uri,
    instance_type=training_instance_type,
    instance_count=1,
    output_path=model_path_bench,
    sagemaker_session=sagemaker_session,
    role=role,
)
xgb_train_deadlift = Estimator(
    image_uri=image_uri,
    instance_type=training_instance_type,
    instance_count=1,
    output_path=model_path_deadlift,
    sagemaker_session=sagemaker_session,
    role=role,
)
xgb_train_bodyweight = Estimator(
    image_uri=image_uri,
    instance_type=training_instance_type,
    instance_count=1,
    output_path=model_path_bodyweight,
    sagemaker_session=sagemaker_session,
    role=role,
)

objective_value = "reg:squarederror"
num_rounds_value = 100
max_depth_value = 9
eta_value = 0.1
verbosity_value = 3
eval_metric_value = "rmse"

# Set Hyperparameters
xgb_train_squat.set_hyperparameters(
    objective=objective_value,
    num_round=num_rounds_value,
    max_depth=max_depth_value,
    eta=eta_value,
    verbosity=verbosity_value,
    eval_metric=eval_metric_value
)
xgb_train_bench.set_hyperparameters(
    objective=objective_value,
    num_round=num_rounds_value,
    max_depth=max_depth_value,
    eta=eta_value,
    verbosity=verbosity_value,
    eval_metric=eval_metric_value
)
xgb_train_deadlift.set_hyperparameters(
    objective=objective_value,
    num_round=num_rounds_value,
    max_depth=max_depth_value,
    eta=eta_value,
    verbosity=verbosity_value,
    eval_metric=eval_metric_value
)
xgb_train_bodyweight.set_hyperparameters(
    objective=objective_value,
    num_round=num_rounds_value,
    max_depth=max_depth_value,
    eta=eta_value,
    verbosity=verbosity_value,
    eval_metric=eval_metric_value
)

Train the model on the dataset.

In [11]:
# Fit model

xgb_train_squat.fit({"train": squat_train_data, "validation" : squat_test_data}, wait=True)
xgb_train_bench.fit({"train": bench_train_data, "validation" : bench_test_data}, wait=True)
xgb_train_deadlift.fit({"train": deadlift_train_data, "validation" : deadlift_test_data}, wait=True)
#xgb_train_bodyweight.fit({"train": bodyweight_train_data, "validation" : bodyweight_test_data}, wait=True)

INFO:sagemaker:Creating training-job with name: sagemaker-xgboost-2023-08-16-01-52-14-225


2023-08-16 01:52:14 Starting - Starting the training job...
2023-08-16 01:52:29 Starting - Preparing the instances for training......
2023-08-16 01:53:23 Downloading - Downloading input data...
2023-08-16 01:54:13 Training - Training image download completed. Training in progress.....[2023-08-16 01:54:24.117 ip-10-0-107-29.us-east-2.compute.internal:7 INFO utils.py:27] RULE_JOB_STOP_SIGNAL_FILENAME: None
INFO:sagemaker-containers:Imported framework sagemaker_xgboost_container.training
INFO:sagemaker-containers:Failed to parse hyperparameter eval_metric value rmse to Json.
Returning the value itself
INFO:sagemaker-containers:Failed to parse hyperparameter objective value reg:squarederror to Json.
Returning the value itself
INFO:sagemaker-containers:No GPUs detected (normal if no gpus installed)
INFO:sagemaker_xgboost_container.training:Running XGBoost Sagemaker in algorithm mode
INFO:root:Determined delimiter of CSV input is ','
INFO:root:Determined delimiter of CSV input is ','
INFO:ro

[01:55:04] INFO: /workspace/src/tree/updater_prune.cc:89: tree pruning end, 784 extra nodes, 0 pruned nodes, max_depth=9
[36]#011train-rmse:21.65800#011validation-rmse:22.73417
[01:55:05] INFO: /workspace/src/tree/updater_prune.cc:89: tree pruning end, 662 extra nodes, 0 pruned nodes, max_depth=9
[37]#011train-rmse:21.56244#011validation-rmse:22.66374
[01:55:06] INFO: /workspace/src/tree/updater_prune.cc:89: tree pruning end, 780 extra nodes, 0 pruned nodes, max_depth=9
[38]#011train-rmse:21.47265#011validation-rmse:22.60491
[01:55:07] INFO: /workspace/src/tree/updater_prune.cc:89: tree pruning end, 840 extra nodes, 0 pruned nodes, max_depth=9
[39]#011train-rmse:21.39155#011validation-rmse:22.55454
[01:55:09] INFO: /workspace/src/tree/updater_prune.cc:89: tree pruning end, 666 extra nodes, 0 pruned nodes, max_depth=9
[40]#011train-rmse:21.32680#011validation-rmse:22.51048
[01:55:10] INFO: /workspace/src/tree/updater_prune.cc:89: tree pruning end, 724 extra nodes, 0 pruned nodes, max_de

[01:55:52] INFO: /workspace/src/tree/updater_prune.cc:89: tree pruning end, 520 extra nodes, 0 pruned nodes, max_depth=9
[84]#011train-rmse:20.25610#011validation-rmse:22.19088
[01:55:53] INFO: /workspace/src/tree/updater_prune.cc:89: tree pruning end, 450 extra nodes, 0 pruned nodes, max_depth=9
[85]#011train-rmse:20.25183#011validation-rmse:22.19053
[01:55:54] INFO: /workspace/src/tree/updater_prune.cc:89: tree pruning end, 424 extra nodes, 0 pruned nodes, max_depth=9
[86]#011train-rmse:20.24152#011validation-rmse:22.18838
[01:55:55] INFO: /workspace/src/tree/updater_prune.cc:89: tree pruning end, 730 extra nodes, 0 pruned nodes, max_depth=9
[87]#011train-rmse:20.22076#011validation-rmse:22.18767
[01:55:55] INFO: /workspace/src/tree/updater_prune.cc:89: tree pruning end, 412 extra nodes, 0 pruned nodes, max_depth=9
[88]#011train-rmse:20.21034#011validation-rmse:22.18416
[01:55:56] INFO: /workspace/src/tree/updater_prune.cc:89: tree pruning end, 452 extra nodes, 0 pruned nodes, max_de

INFO:sagemaker:Creating training-job with name: sagemaker-xgboost-2023-08-16-01-56-58-639


...

[02:00:17] INFO: /workspace/src/tree/updater_prune.cc:89: tree pruning end, 210 extra nodes, 0 pruned nodes, max_depth=9
[78]#011train-rmse:13.22401#011validation-rmse:14.90573
[02:00:18] INFO: /workspace/src/tree/updater_prune.cc:89: tree pruning end, 510 extra nodes, 0 pruned nodes, max_depth=9
[79]#011train-rmse:13.21101#011validation-rmse:14.90406
[02:00:18] INFO: /workspace/src/tree/updater_prune.cc:89: tree pruning end, 234 extra nodes, 0 pruned nodes, max_depth=9
[80]#011train-rmse:13.20407#011validation-rmse:14.90456
[02:00:19] INFO: /workspace/src/tree/updater_prune.cc:89: tree pruning end, 514 extra nodes, 0 pruned nodes, max_depth=9
[81]#011train-rmse:13.19189#011validation-rmse:14.90186
[02:00:20] INFO: /workspace/src/tree/updater_prune.cc:89: tree pruning end, 592 extra nodes, 0 pruned nodes, max_depth=9
[82]#011train-rmse:13.18111#011validation-rmse:14.90414
[02:00:21] INFO: /workspace/src/tree/updater_prune.cc:89: tree pruning end, 524 extra nodes, 0 pruned nodes, max_de

INFO:sagemaker:Creating training-job with name: sagemaker-xgboost-2023-08-16-02-01-12-332


Training seconds: 172
Billable seconds: 172
2023-08-16 02:01:12 Starting - Starting the training job...
2023-08-16 02:01:26 Starting - Preparing the instances for training......
2023-08-16 02:02:26 Downloading - Downloading input data...
2023-08-16 02:03:10 Training - Downloading the training image...
2023-08-16 02:03:16 Training - Training image download completed. Training in progress..[2023-08-16 02:03:26.354 ip-10-0-80-22.us-east-2.compute.internal:7 INFO utils.py:27] RULE_JOB_STOP_SIGNAL_FILENAME: None
INFO:sagemaker-containers:Imported framework sagemaker_xgboost_container.training
INFO:sagemaker-containers:Failed to parse hyperparameter eval_metric value rmse to Json.
Returning the value itself
INFO:sagemaker-containers:Failed to parse hyperparameter objective value reg:squarederror to Json.
Returning the value itself
INFO:sagemaker-containers:No GPUs detected (normal if no gpus installed)
INFO:sagemaker_xgboost_container.training:Running XGBoost Sagemaker in algorithm mode
INFO

[02:03:56] INFO: /workspace/src/tree/updater_prune.cc:89: tree pruning end, 812 extra nodes, 0 pruned nodes, max_depth=9
[29]#011train-rmse:20.15879#011validation-rmse:21.06243
[02:03:57] INFO: /workspace/src/tree/updater_prune.cc:89: tree pruning end, 762 extra nodes, 0 pruned nodes, max_depth=9
[30]#011train-rmse:19.80216#011validation-rmse:20.73756
[02:03:58] INFO: /workspace/src/tree/updater_prune.cc:89: tree pruning end, 790 extra nodes, 0 pruned nodes, max_depth=9
[31]#011train-rmse:19.50287#011validation-rmse:20.46798
[02:03:59] INFO: /workspace/src/tree/updater_prune.cc:89: tree pruning end, 856 extra nodes, 0 pruned nodes, max_depth=9
[32]#011train-rmse:19.24714#011validation-rmse:20.24480
[02:04:00] INFO: /workspace/src/tree/updater_prune.cc:89: tree pruning end, 870 extra nodes, 0 pruned nodes, max_depth=9
[33]#011train-rmse:19.02322#011validation-rmse:20.05546
[02:04:01] INFO: /workspace/src/tree/updater_prune.cc:89: tree pruning end, 808 extra nodes, 0 pruned nodes, max_de

[02:04:38] INFO: /workspace/src/tree/updater_prune.cc:89: tree pruning end, 520 extra nodes, 0 pruned nodes, max_depth=9
[72]#011train-rmse:17.42321#011validation-rmse:19.12047
[02:04:39] INFO: /workspace/src/tree/updater_prune.cc:89: tree pruning end, 658 extra nodes, 0 pruned nodes, max_depth=9
[73]#011train-rmse:17.41198#011validation-rmse:19.12033
[02:04:40] INFO: /workspace/src/tree/updater_prune.cc:89: tree pruning end, 262 extra nodes, 0 pruned nodes, max_depth=9
[74]#011train-rmse:17.40534#011validation-rmse:19.12002
[02:04:40] INFO: /workspace/src/tree/updater_prune.cc:89: tree pruning end, 162 extra nodes, 0 pruned nodes, max_depth=9
[75]#011train-rmse:17.40253#011validation-rmse:19.12091
[02:04:41] INFO: /workspace/src/tree/updater_prune.cc:89: tree pruning end, 606 extra nodes, 0 pruned nodes, max_depth=9
[76]#011train-rmse:17.39400#011validation-rmse:19.11983
[02:04:42] INFO: /workspace/src/tree/updater_prune.cc:89: tree pruning end, 274 extra nodes, 0 pruned nodes, max_de

## Deployment

After training the model, retrieve the model artifacts so that we can deploy the model to an endpoint.

In [20]:
# Retrieve model data from training job
squat_model_artifacts = xgb_train_squat.model_data
bench_model_artifacts = xgb_train_bench.model_data
deadlift_model_artifacts = xgb_train_deadlift.model_data
#bodyweight_model_artifacts = xgb_train_bodyweight.model_data

TypeError: join() argument must be str, bytes, or os.PathLike object, not 'NoneType'

### Model Creation
Create a model by providing your model artifacts, the container image URI, environment variables for the container (if applicable), a model name, and the SageMaker IAM role.

In [13]:
squat_model_name = "jakeshape-performance-predictor-squat-model"
bench_model_name = "jakeshape-performance-predictor-bench-model"
deadlift_model_name = "jakeshape-performance-predictor-deadlift-model"
bodyweight_model_name = "jakeshape-performance-predictor-bodyweight-model"

create_model_response_squat = client.create_model(
    ModelName=squat_model_name,
    Containers=[
        {
            "Image": image_uri,
            "Mode": "SingleModel",
            "ModelDataUrl": squat_model_artifacts,
        }
    ],
    ExecutionRoleArn=role,
)

print("Model Arn: " + create_model_response_squat["ModelArn"])

create_model_response_bench = client.create_model(
    ModelName=bench_model_name,
    Containers=[
        {
            "Image": image_uri,
            "Mode": "SingleModel",
            "ModelDataUrl": bench_model_artifacts,
        }
    ],
    ExecutionRoleArn=role,
)

print("Model Arn: " + create_model_response_bench["ModelArn"])

create_model_response_deadlift = client.create_model(
    ModelName=deadlift_model_name,
    Containers=[
        {
            "Image": image_uri,
            "Mode": "SingleModel",
            "ModelDataUrl": deadlift_model_artifacts,
        }
    ],
    ExecutionRoleArn=role,
)

print("Model Arn: " + create_model_response_deadlift["ModelArn"])
'''
create_model_response_bodyweight = client.create_model(
    ModelName=bodyweight_model_name,
    Containers=[
        {
            "Image": image_uri,
            "Mode": "SingleModel",
            "ModelDataUrl": bodyweight_model_artifacts,
        }
    ],
    ExecutionRoleArn=role,
)

print("Model Arn: " + create_model_response_bodyweight["ModelArn"])'''

Model Arn: arn:aws:sagemaker:us-east-2:544485497213:model/jakeshape-performance-predictor-squat-model
Model Arn: arn:aws:sagemaker:us-east-2:544485497213:model/jakeshape-performance-predictor-bench-model
Model Arn: arn:aws:sagemaker:us-east-2:544485497213:model/jakeshape-performance-predictor-deadlift-model


'\ncreate_model_response_bodyweight = client.create_model(\n    ModelName=bodyweight_model_name,\n    Containers=[\n        {\n            "Image": image_uri,\n            "Mode": "SingleModel",\n            "ModelDataUrl": bodyweight_model_artifacts,\n        }\n    ],\n    ExecutionRoleArn=role,\n)\n\nprint("Model Arn: " + create_model_response_bodyweight["ModelArn"])'

### Endpoint Configuration Creation

This is where you can adjust the <b>Serverless Configuration</b> for your endpoint. The current max concurrent invocations for a single endpoint, known as `MaxConcurrency`, can be any value from <b>1 to 200</b>, and `MemorySize` can be any of the following: <b>1024 MB, 2048 MB, 3072 MB, 4096 MB, 5120 MB, or 6144 MB</b>.

In [14]:
epc_squat_name = "jakeshape-performance-predictor-squat-endpoint-config"
epc_bench_name = "jakeshape-performance-predictor-bench-endpoint-config"
epc_deadlift_name = "jakeshape-performance-predictor-deadlift-endpoint-config"
epc_bodyweight_name = "jakeshape-performance-predictor-bodyweight-endpoint-config"


endpoint_config_response_squat = client.create_endpoint_config(
    EndpointConfigName=epc_squat_name,
    ProductionVariants=[
        {
            "VariantName": "squatVariant",
            "ModelName": squat_model_name,
            "ServerlessConfig": {
                "MemorySizeInMB": 3072,
                "MaxConcurrency": 2,
            },
        },
    ],
)

print("Endpoint Configuration Arn: " + endpoint_config_response_squat["EndpointConfigArn"])

endpoint_config_response_bench = client.create_endpoint_config(
    EndpointConfigName=epc_bench_name,
    ProductionVariants=[
        {
            "VariantName": "benchVariant",
            "ModelName": bench_model_name,
            "ServerlessConfig": {
                "MemorySizeInMB": 3072,
                "MaxConcurrency": 2,
            },
        },
    ],
)

print("Endpoint Configuration Arn: " + endpoint_config_response_bench["EndpointConfigArn"])

endpoint_config_response_deadlift = client.create_endpoint_config(
    EndpointConfigName=epc_deadlift_name,
    ProductionVariants=[
        {
            "VariantName": "deadliftVariant",
            "ModelName": deadlift_model_name,
            "ServerlessConfig": {
                "MemorySizeInMB": 3072,
                "MaxConcurrency": 2,
            },
        },
    ],
)

print("Endpoint Configuration Arn: " + endpoint_config_response_deadlift["EndpointConfigArn"])

endpoint_config_response_bodyweight = client.create_endpoint_config(
    EndpointConfigName=epc_bodyweight_name,
    ProductionVariants=[
        {
            "VariantName": "bodyweightVariant",
            "ModelName": bodyweight_model_name,
            "ServerlessConfig": {
                "MemorySizeInMB": 3072,
                "MaxConcurrency": 2,
            },
        },
    ],
)

print("Endpoint Configuration Arn: " + endpoint_config_response_bodyweight["EndpointConfigArn"])

Endpoint Configuration Arn: arn:aws:sagemaker:us-east-2:544485497213:endpoint-config/jakeshape-performance-predictor-squat-endpoint-config
Endpoint Configuration Arn: arn:aws:sagemaker:us-east-2:544485497213:endpoint-config/jakeshape-performance-predictor-bench-endpoint-config
Endpoint Configuration Arn: arn:aws:sagemaker:us-east-2:544485497213:endpoint-config/jakeshape-performance-predictor-deadlift-endpoint-config


'\nendpoint_config_response_bodyweight = client.create_endpoint_config(\n    EndpointConfigName=epc_bodyweight_name,\n    ProductionVariants=[\n        {\n            "VariantName": "bodyweightVariant",\n            "ModelName": bodyweight_model_name,\n            "ServerlessConfig": {\n                "MemorySizeInMB": 3072,\n                "MaxConcurrency": 2,\n            },\n        },\n    ],\n)\n\nprint("Endpoint Configuration Arn: " + endpoint_config_response_bodyweight["EndpointConfigArn"])'

### Serverless Endpoint Creation
Now that we have an endpoint configuration, we can create a serverless endpoint and deploy our model to it. When creating the endpoint, provide the name of your endpoint configuration and a name for the new endpoint.

In [15]:
squat_endpoint_name = "jakeshape-performance-predictor-squat-endpoint"
bench_endpoint_name = "jakeshape-performance-predictor-bench-endpoint"
deadlift_endpoint_name = "jakeshape-performance-predictor-deadlift-endpoint"
bodyweight_endpoint_name = "jakeshape-performance-predictor-bodyweight-endpoint"

create_endpoint_response_squat = client.create_endpoint(
    EndpointName=squat_endpoint_name,
    EndpointConfigName=epc_squat_name,
)

print("Endpoint Arn: " + create_endpoint_response_squat["EndpointArn"])

create_endpoint_response_bench = client.create_endpoint(
    EndpointName=bench_endpoint_name,
    EndpointConfigName=epc_bench_name,
)

print("Endpoint Arn: " + create_endpoint_response_bench["EndpointArn"])

create_endpoint_response_deadlift = client.create_endpoint(
    EndpointName=deadlift_endpoint_name,
    EndpointConfigName=epc_deadlift_name,
)

print("Endpoint Arn: " + create_endpoint_response_deadlift["EndpointArn"])

create_endpoint_response_bodyweight = client.create_endpoint(
    EndpointName=bodyweight_endpoint_name,
    EndpointConfigName=epc_bodyweight_name,
)

print("Endpoint Arn: " + create_endpoint_response_bodyweight["EndpointArn"])

Endpoint Arn: arn:aws:sagemaker:us-east-2:544485497213:endpoint/jakeshape-performance-predictor-squat-endpoint
Endpoint Arn: arn:aws:sagemaker:us-east-2:544485497213:endpoint/jakeshape-performance-predictor-bench-endpoint
Endpoint Arn: arn:aws:sagemaker:us-east-2:544485497213:endpoint/jakeshape-performance-predictor-deadlift-endpoint


'\ncreate_endpoint_response_bodyweight = client.create_endpoint(\n    EndpointName=bodyweight_endpoint_name,\n    EndpointConfigName=epc_bodyweight_name,\n)\n\nprint("Endpoint Arn: " + create_endpoint_response_bodyweight["EndpointArn"])'

### Hyperparameter Tuning Exploration

In [75]:
from sagemaker.tuner import IntegerParameter, ContinuousParameter, HyperparameterTuner

hyperparameter_ranges = {
    "eta": ContinuousParameter(0.01, 0.2),
    "max_depth": IntegerParameter(3, 12),
}
objective_metric_name = "validation:rmse"

# Configure Training Estimator
xgb_train_squat_tuning = Estimator(
    image_uri=image_uri,
    instance_type=training_instance_type,
    instance_count=1,
    output_path=f"s3://{s3_prefix}/{squat_path}/tuning",
    sagemaker_session=sagemaker_session,
    role=role,
)

tuner = HyperparameterTuner(
    xgb_train_squat_tuning,
    objective_metric_name,
    hyperparameter_ranges,
    max_jobs=10,
    max_parallel_jobs=2,
    objective_type="Minimize",
    max_runtime_in_seconds=1800,
    strategy="Random"
)
data_channels = {
    "train": squat_train_data,
    "validation": squat_test_data,
}

tuner.fit(data_channels)



INFO:sagemaker:Creating hyperparameter tuning job with name: sagemaker-xgboost-230814-1825


................................................*


UnexpectedStatusException: Error for HyperParameterTuning job sagemaker-xgboost-230814-1825: Failed. Reason: No training job succeeded after 5 attempts. For additional details, please take a look at the training job failures by listing training jobs for the hyperparameter tuning job.